In [438]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random


# Get HPI data for ALL MSA in the us:

## go to this site: 
https://www.fhfa.gov/DataTools/Downloads/Pages/House-Price-Index-Datasets.aspx#qat

### Look under Quarterly Data - > All-Transactions Indexes (Estimated using Sales Prices and Appraisal Data)-> Metropolitan Statistical Areas and Divisions (Not Seasonally Adjusted)    

and download the CSV.
this is the HPI for all of the us MSA's - this should include your target + the full MSA list.

In [336]:
hpi_info = pd.read_csv("new_datasets/HPI_AT_metro.csv",names=['Place', 'Hpi', 'Year','Quarter','Adj','P_Chane'])
hpi_info.insert(1,'State',[place.split(',')[1] for place in hpi_info['Place']])
hpi_info.insert(2,'MSA',[place.split(',')[0] for place in hpi_info['Place']])
q2t = {1:3,2:6,3:9,4:12}
hpi_info.insert(6,'month',[q2t[q] for q in hpi_info['Quarter']])
hpi_info['short_date'] = [(f"{hpi_info['Year'][row]}-{hpi_info['month'][row]}") for row in hpi_info.index]

hpi_data = hpi_info.loc[hpi_info['Year']>=1990].reset_index()
hpi_data = hpi_data.drop('index',axis='columns')

In [ ]:
hpi_data

# get FRED info

### Getting an api key for FRED:
1. read - https://pypi.org/project/fredapi/
2. go to here: https://research.stlouisfed.org/docs/api/api_key.html and apply for an api key
3. Enter your api key below

In [5]:
from fredapi import Fred
import requests
import time
import os
from collections import Counter


api_key = 'your_api_key_here'
fred = Fred(api_key=api_key)

# GET all TAGS:

In [7]:
tags = requests.get(f'https://api.stlouisfed.org/fred/tags?api_key={api_key}&file_type=json&limit=1000').json()
print(len(tags['tags']))

1000


In [11]:
import concurrent.futures

tag_names = [tag['name'] for tag in tags['tags'] if tag['name']]

seen_tags = set()
seen_series = set()
all_series = []

def get_tag_series(tag):
    try:
        resp = requests.get(f'https://api.stlouisfed.org/fred/tags/series?tag_names={tag}&api_key={api_key}&file_type=json').json()
    except:
        resp = {}
    return tag, resp


# Build all_series - this is a list with All of Fred TS data !
this will take time - however it is parallel programming

In [13]:
for i in range(30):
    tags_to_run = [tag for tag in tag_names if tag not in seen_tags]
    if tags_to_run:
        print(f'running {len(tags_to_run)} tags')
        with concurrent.futures.ThreadPoolExecutor(max_workers=1000) as pool:        
            res = list(pool.map(get_tag_series, tags_to_run[:100]))
    else:
        continue

    print(f'done chunk: {len(res)}')
    for tag, i in res:
        if 'seriess' not in i:
            print('problem in tag' , tag, i)
            continue
        else:
            seen_tags.add(tag)

        for series in i['seriess']:
            if series['id'] not in seen_series:
                seen_series.add(series['id'])
                all_series.append(series)


    print(f'so far: {len(all_series)} seriess')
    time.sleep(30)

running 1000 tags
done chunk: 100
so far: 50833 seriess
running 900 tags
done chunk: 100
so far: 105741 seriess
running 800 tags
done chunk: 100
problem in tag services-providing {'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag europe {'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variable tag_names does not exist.'}
problem in tag 16 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "16 years  " for variable tag_names does not exist.'}
problem in tag foreign {'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
problem in tag payrolls {'error_code': 429, 'error_message': 'Too Many Requests.  Exceeded Rate Limit'}
problem in tag 15 

running 782 tags
done chunk: 100
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variable tag_names does not exist.'}
problem in tag 16 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "16 years  " for variable tag_names does not exist.'}
so far: 161381 seriess
running 685 tags
done chunk: 100
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variable tag_names does not exist.'}
problem in tag 16 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "16 years  " for variable tag_names does not exist.'}
so far: 216076 seriess
running 588 tags
done chunk: 100
problem in tag 25 years + {

running 571 tags
done chunk: 100
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variable tag_names does not exist.'}
problem in tag 16 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "16 years  " for variable tag_names does not exist.'}
so far: 273368 seriess
running 474 tags
done chunk: 100
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variable tag_names does not exist.'}
problem in tag 16 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "16 years  " for variable tag_names does not exist.'}
problem in tag r&d {'error_code': 400, 'error_message': 'Bad Request.  Value "r" fo

running 361 tags
done chunk: 100
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variable tag_names does not exist.'}
problem in tag 16 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "16 years  " for variable tag_names does not exist.'}
problem in tag r&d {'error_code': 400, 'error_message': 'Bad Request.  Value "r" for variable tag_names does not exist.'}
problem in tag 65-years + {'error_code': 400, 'error_message': 'Bad Request.  Value "65-years  " for variable tag_names does not exist.'}
so far: 364838 seriess
running 266 tags
done chunk: 100
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " fo

running 249 tags
done chunk: 100
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variable tag_names does not exist.'}
problem in tag 16 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "16 years  " for variable tag_names does not exist.'}
problem in tag r&d {'error_code': 400, 'error_message': 'Bad Request.  Value "r" for variable tag_names does not exist.'}
problem in tag 65-years + {'error_code': 400, 'error_message': 'Bad Request.  Value "65-years  " for variable tag_names does not exist.'}
so far: 396495 seriess
running 154 tags
done chunk: 100
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " fo

running 138 tags
done chunk: 100
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variable tag_names does not exist.'}
problem in tag 16 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "16 years  " for variable tag_names does not exist.'}
problem in tag r&d {'error_code': 400, 'error_message': 'Bad Request.  Value "r" for variable tag_names does not exist.'}
problem in tag 65-years + {'error_code': 400, 'error_message': 'Bad Request.  Value "65-years  " for variable tag_names does not exist.'}
so far: 420604 seriess
running 43 tags
done chunk: 43
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for 

running 5 tags
done chunk: 5
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variable tag_names does not exist.'}
problem in tag 16 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "16 years  " for variable tag_names does not exist.'}
problem in tag r&d {'error_code': 400, 'error_message': 'Bad Request.  Value "r" for variable tag_names does not exist.'}
problem in tag 65-years + {'error_code': 400, 'error_message': 'Bad Request.  Value "65-years  " for variable tag_names does not exist.'}
so far: 426904 seriess
running 5 tags
done chunk: 5
problem in tag 25 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "25 years  " for variable tag_names does not exist.'}
problem in tag 15 years + {'error_code': 400, 'error_message': 'Bad Request.  Value "15 years  " for variab

# Build a DF

In [440]:
df = pd.DataFrame(all_series)
#now you should have all the timeseries in a df

In [441]:
df.head(2)

id realtime_start realtime_end  \
0  00XALCATM086NEST     2021-05-09   2021-05-09   
1  00XALCBEM086NEST     2021-05-09   2021-05-09   

                                               title observation_start  \
0  Harmonized Index of Consumer Prices: Overall I...        1999-12-01   
1  Harmonized Index of Consumer Prices: Overall I...        1998-12-01   

  observation_end frequency frequency_short           units     units_short  \
0      2021-03-01   Monthly               M  Index 2015=100  Index 2015=100   
1      2021-03-01   Monthly               M  Index 2015=100  Index 2015=100   

       seasonal_adjustment seasonal_adjustment_short            last_updated  \
0  Not Seasonally Adjusted                       NSA  2021-04-16 07:11:21-05   
1  Not Seasonally Adjusted                       NSA  2021-04-16 07:16:28-05   

   popularity  group_popularity  \
0           1                 1   
1           1                 1   

                                               notes  
0  The Harmonized Index of Consumer Prices catego...  
1  The Harmonized Index of Consumer Prices catego...

In [442]:
df.shape

(426904, 16)

# filtering

In [444]:
#סינון לפי סוג אזור
def is_series_valid(title):
    title = title.lower()
    return ('cbsa' in title or 'msa' in title or 'necta' in title)# and ('dallas' in title or 'bridgeport' in title) 

#סינון לפי שנת התחלה
from datetime import datetime

def is_series_valid_time_start(start_date):
    start_year = datetime.strptime(start_date, '%Y-%m-%d').year
    return (start_year <= 2010)

#סינון לפי שנת סיום
def is_series_valid_time_end(end_date):
    end_year = datetime.strptime(end_date, '%Y-%m-%d').year
    return (end_year > 2015)


msa_series = df[df.title.apply(is_series_valid)]
msa_series = msa_series[msa_series.observation_start.apply(is_series_valid_time_start)]
msa_series = msa_series[msa_series.observation_end.apply(is_series_valid_time_end)]
msa_series = msa_series.reset_index()
for row in msa_series.index:
    if ", in (MSA)" in msa_series['title'][row]:
        msa_series['title'][row] = msa_series['title'][row].replace(", in (MSA)",", IN (MSA)")

<ipython-input-444-8fb654586b8e>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msa_series['title'][row] = msa_series['title'][row].replace(", in (MSA)",", IN (MSA)")


In [445]:
series_list = []
places_list = []

#תוספת הפרדה של שם סדרה והמיקום
for name in msa_series['title']:  
    if " in " in name:
        if "Change in" in name and " for " in name:
                series_list.append(name.split(' for ')[0])
                places_list.append(name.split(' for ')[1])
        else:
            if name.count(' in ') == 1:
                series_list.append(name.split(' in ')[0])
                places_list.append(name.split(' in ')[1])
            else:
                series_list.append(" in ".join(name.split(' in ')[:-1]))
                places_list.append(name.split(' in ')[-1])
    elif " for " in name:
        if name.count(' for ') == 1:
            series_list.append(name.split(' for ')[0])
            places_list.append(name.split(' for ')[1])
        else:
            series_list.append(" for ".join(name.split(' for ')[:-1]))
            places_list.append(name.split(' for ')[-1])
    else:
        series_list.append(name)
        places_list.append(None)

p = {"Urban Alaska (CBSA)":"Urban Alaska, AK (CBSA)",
     "Seattle-Tacoma-Bellevue WA (CBSA)":"Seattle-Tacoma-Bellevue, WA (CBSA)",
     "Urban Hawaii (CBSA)":"Urban Hawaii, HI (CBSA)"}
for ind,place in enumerate(places_list):
    if place in p.keys():
        places_list[ind] = p[place]        
        
msa_series['series'] = series_list
msa_series['place_title'] = places_list
msa_series['state'] = [p.split("(")[0].split(',')[1] for p in msa_series['place_title']]
msa_series['msa'] = [p.split("(")[0].split(',')[0] for p in msa_series['place_title']]


In [446]:
msa_series.shape

(36237, 21)

In [447]:
msa_series.head(3)

index            id realtime_start realtime_end  \
0   2408   ABIL148EDUH     2021-05-09   2021-05-09   
1   2409  ABIL148EDUHN     2021-05-09   2021-05-09   
2   2410   ABIL148FIRE     2021-05-09   2021-05-09   

                                               title observation_start  \
0  All Employees: Education and Health Services i...        1990-01-01   
1  All Employees: Education and Health Services i...        1990-01-01   
2  All Employees: Financial Activities in Abilene...        1990-01-01   

  observation_end frequency frequency_short                 units  \
0      2021-03-01   Monthly               M  Thousands of Persons   
1      2021-03-01   Monthly               M  Thousands of Persons   
2      2021-03-01   Monthly               M  Thousands of Persons   

         units_short      seasonal_adjustment seasonal_adjustment_short  \
0  Thous. of Persons      Seasonally Adjusted                        SA   
1  Thous. of Persons  Not Seasonally Adjusted                       NSA   
2  Thous. of Persons      Seasonally Adjusted                        SA   

             last_updated  popularity  group_popularity  \
0  2021-04-16 16:53:47-05           1                 1   
1  2021-04-16 15:21:34-05           1                 1   
2  2021-04-16 16:53:47-05           1                 1   

                                               notes  \
0  The Federal Reserve Bank of St. Louis seasonal...   
1                                                NaN   
2  The Federal Reserve Bank of St. Louis seasonal...   

                                         series        place_title state  \
0  All Employees: Education and Health Services  Abilene, TX (MSA)   TX    
1  All Employees: Education and Health Services  Abilene, TX (MSA)   TX    
2           All Employees: Financial Activities  Abilene, TX (MSA)   TX    

       msa  
0  Abilene  
1  Abilene  
2  Abilene

## find relevant serieses

In [448]:
msa_series_groupby = msa_series.groupby(['series']).count().sort_values(['title'],ascending=False) 
msa_series_groupby

index    id  \
series                                                            
All Employees: Total Nonfarm                         1177  1177   
All Employees: Mining, Logging, and Construction      875   875   
All Employees: Leisure and Hospitality                841   841   
Civilian Labor Force                                  792   792   
Unemployment Rate                                     774   774   
All Employees: Government                             753   753   
All Employees: Trade, Transportation, and Utili...    741   741   
Average Weekly Wages for Employees in Total Cov...    704   704   
All Employees: Transportation and Utilities: Tr...    694   694   
All Employees: Manufacturing                          692   692   
All Employees: Professional and Business Services     683   683   
All Employees: Education and Health Services          675   675   
All Employees: Total Private                          669   669   
All Employees: Service-Providing                      666   666   
All Employees: Private Service Providing              666   666   
All Employees: Goods Producing                        666   666   
All Employees: Information                            644   644   
All Employees: Financial Activities                   643   643   
Average Weekly Earnings of All Employees: Total...    630   630   
Average Hourly Earnings of All Employees: Total...    630   630   
Average Weekly Hours of All Employees: Total Pr...    627   627   
All Employees: Other Services                         621   621   
Average Weekly Wages for Employees in State Gov...    614   614   
Average Weekly Wages for Employees in Local Gov...    609   609   
All Employees: Retail Trade                           608   608   
Unemployed Persons                                    528   528   
Employed Persons                                      527   527   
All Employees: Wholesale Trade                        521   521   
Average Weekly Wages for Employees in Federal G...    500   500   
Total Quarterly Wages                                 500   500   
Average Weekly Wages for Employees in Private E...    498   498   
All Employees: Government: State Government           423   423   
All Employees: Government: Local Government           422   422   
All Employees: Government: Federal Government         408   408   
All-Transactions House Price Index                    407   407   
Implicit Regional Price Deflator                      381   381   
Per Capita Personal Income                            328   328   
Resident Population                                   266   266   
Total Gross Domestic Product                          234   234   
Total Per Capita Real Gross Domestic Product          233   233   
New Private Housing Units Authorized by Buildin...    226   226   
All Employees: Retail Trade: General Merchandis...    213   213   
Total Quantity Indexes for Real GDP                   213   213   
All Employees: Manufacturing: Durable Goods           211   211   
Total Real Gross Domestic Product                     210   210   
Regional Price Parities: Services: Rents              209   209   
Regional Price Parities: Services: Other              209   209   
Regional Price Parities: All Items                    209   209   
Real Personal Income                                  209   209   
Regional Price Parities: Goods                        209   209   
Real Per Capita Personal Income                       209   209   
All Employees: State Government                       193   193   
All Employees: Local Government                       193   193   
All Employees: Federal Government                     186   186   
All Employees: Manufacturing: Non-Durable Goods       184   184   
All Employees: Financial Activities: Finance an...    159   159   
All Employees: Education and Health Services: H...    154   154   
All Employees: Construction: Specialty Trade Co...    150   150   
All Employees: Government: Local

In [453]:
serieses_to_search = msa_series_groupby[['id']][msa_series_groupby>100].dropna(axis='rows')
print(f'\ntotal serieses to search: {len(serieses_to_search.index)}')

serieses_to_search


total serieses to search: 67


id
series                                                    
All Employees: Total Nonfarm                        1177.0
All Employees: Mining, Logging, and Construction     875.0
All Employees: Leisure and Hospitality               841.0
Civilian Labor Force                                 792.0
Unemployment Rate                                    774.0
All Employees: Government                            753.0
All Employees: Trade, Transportation, and Utili...   741.0
Average Weekly Wages for Employees in Total Cov...   704.0
All Employees: Transportation and Utilities: Tr...   694.0
All Employees: Manufacturing                         692.0
All Employees: Professional and Business Services    683.0
All Employees: Education and Health Services         675.0
All Employees: Total Private                         669.0
All Employees: Service-Providing                     666.0
All Employees: Private Service Providing             666.0
All Employees: Goods Producing                       666.0
All Employees: Information                           644.0
All Employees: Financial Activities                  643.0
Average Weekly Earnings of All Employees: Total...   630.0
Average Hourly Earnings of All Employees: Total...   630.0
Average Weekly Hours of All Employees: Total Pr...   627.0
All Employees: Other Services                        621.0
Average Weekly Wages for Employees in State Gov...   614.0
Average Weekly Wages for Employees in Local Gov...   609.0
All Employees: Retail Trade                          608.0
Unemployed Persons                                   528.0
Employed Persons                                     527.0
All Employees: Wholesale Trade                       521.0
Average Weekly Wages for Employees in Federal G...   500.0
Total Quarterly Wages                                500.0
Average Weekly Wages for Employees in Private E...   498.0
All Employees: Government: State Government          423.0
All Employees: Government: Local Government          422.0
All Employees: Government: Federal Government        408.0
All-Transactions House Price Index                   407.0
Implicit Regional Price Deflator                     381.0
Per Capita Personal Income                           328.0
Resident Population                                  266.0
Total Gross Domestic Product                         234.0
Total Per Capita Real Gross Domestic Product         233.0
New Private Housing Units Authorized by Buildin...   226.0
All Employees: Retail Trade: General Merchandis...   213.0
Total Quantity Indexes for Real GDP                  213.0
All Employees: Manufacturing: Durable Goods          211.0
Total Real Gross Domestic Product                    210.0
Regional Price Parities: Services: Rents             209.0
Regional Price Parities: Services: Other             209.0
Regional Price Parities: All Items                   209.0
Real Personal Income                                 209.0
Regional Price Parities: Goods                       209.0
Real Per Capita Personal Income                      209.0
All Employees: State Government                      193.0
All Employees: Local Government                      193.0
All Employees: Federal Government                    186.0
All Employees: Manufacturing: Non-Durable Goods      184.0
All Employees: Financial Activities: Finance an...   159.0
All Employees: Education and Health Services: H...   154.0
All Employees: Construction: Specialty Trade Co...   150.0
All Employees: Government: Local Government Edu...   145.0
All Employees: Professional and Business Servic...   134.0
All Employees: Mining and Logging                    133.0
All Employees: Leisure and Hospitality: Accommo...   128.0
All Employees: Retail Trade: Food and Beverage ...   120.0
All Employees: Leisure and Hospitality: Food Se...   119.0
New Private Housing Units Authorized by Buildin...   110.0
All Employees: Construction                          102.0
All Employees: Education and Health Services

In [454]:
#get all relevant serieses ids
search_ids = []
for row in msa_series.index:
    if msa_series['series'][row] in list(serieses_to_search.index):
        search_ids.append(msa_series['id'][row])

In [455]:
len(search_ids)

28509

In [456]:
#save metadata
base_dir = 'fred_datasets'
if not os.path.exists(base_dir):
    os.mkdir(base_dir)
df_to_download = msa_series.loc[msa_series['id'].isin(search_ids)]


df.to_csv(os.path.join(base_dir, 'total_metadata.csv'), index=False)
df_to_download.to_csv(os.path.join(base_dir, 'relevant_metadata.csv'), index=False)

In [457]:
#separate ids by series
serieses_ids = {}
for series in serieses_to_search.index:
    series_datas = df_to_download.loc[df_to_download['series']==series]
    
    sa = series_datas.loc[series_datas['seasonal_adjustment_short']=='SA']
    sa_ids_list = list(sa['id'])
    if len(sa_ids_list)>0:
        serieses_ids[f"{series}_Seasonally Adjusted"] = np.unique(sa_ids_list)
    
    nsa = series_datas.loc[series_datas['seasonal_adjustment_short']=='NSA']
    nsa_ids_list = list(nsa['id'])
    if len(nsa_ids_list)>0:
        serieses_ids[f"{series}_Not Seasonally Adjusted"] = np.unique(nsa_ids_list)
        
    ssa = series_datas.loc[series_datas['seasonal_adjustment_short']=='SSA']
    ssa_ids_list = list(ssa['id'])
    if len(ssa_ids_list)>0:
        serieses_ids[f"{series}_Smoothed Seasonally Adjusted"] = np.unique(ssa_ids_list)
    

# download the relevant serieses

In [ ]:
print(f'downloading {len(serieses_ids)} seriese \n')


def download_series(series_name):
    print(f"now downloading: {series_name}")
    ids = serieses_ids[series_name]
    i=0
    for current_id in tqdm(ids):
        try:
            info = fred.get_series(current_id)
        except:
            print(f'problem in time series: {current_id}')
            continue
             
        data = pd.DataFrame(info,columns=[series_name]).reset_index()
        data = data.rename({'index':'Date'}, axis=1)
        data.insert(0,"State",df_to_download.loc[df_to_download['id']==current_id]['state'].item())
        data.insert(1,"MSA",df_to_download.loc[df_to_download['id']==current_id]['msa'].item())
        if freq =='M':
            data[f'{series_name}_1_Month_change'] = pd.DataFrame(data[series_name]).pct_change(periods=1, limit=None, freq=None)
            data[f'{series_name}_1_Year_change'] = pd.DataFrame(data[series_name]).pct_change(periods=12, limit=None, freq=None)
        elif freq =='A':
            data[f'{series_name}_1_Year_change'] = pd.DataFrame(data[series_name]).pct_change(periods=1, limit=None, freq=None)
        elif freq =='Q':
            data[f'{series_name}_1_Quarter_change'] = pd.DataFrame(data[series_name]).pct_change(periods=1, limit=None, freq=None)
            data[f'{series_name}_1_Year_change'] = pd.DataFrame(data[series_name]).pct_change(periods=4, limit=None, freq=None)  
        data['short_date'] = [f'{x.year}-{x.month}' for x in data['Date']]
        
        if i==1:
            final_df = pd.concat([final_df, data], ignore_index=True)
        else:
            final_df = data
            i=1
    
    final_df = final_df.reset_index()
    final_df = final_df.drop('index',axis='columns')
    final_df.to_csv(f'{series_name}_data.csv')
    time.sleep(random.randint(5,10))
    return 'ok'



for series_name in serieses_ids.keys():
    download_series(series_name)
   

  0%|          | 0/650 [00:00<?, ?it/s]

downloading 120 seriese 

now downloading: All Employees: Total Nonfarm_Seasonally Adjusted


 51%|█████     | 330/650 [05:17<04:16,  1.25it/s]